<a href="https://colab.research.google.com/github/plackie/planets/blob/master/NDS_Transcripts_Scraping_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Documentation and usage

This Colab notebook was written by **Hiromichi Ueda** '21 (DataSquad) in February 2021.

Revised by:
* **Helen Du** '22 (DataSquad) **Isabella Cha** '23 (DataSquad) in June 2021
* **Cathy Duan** '25 (DataSquad), **Graham Gordon** '25 (DataSquad), **Charles Nykamp** '25 (DataSquad), and **Nina Sun** '23 (DataSquad) in October 2022

**Aaron Bronstone** '24 and **Serafin Patino** '24 (DataSquad)

Last execution in October 2022 on Google Colab.

This script takes an .xls file from NDS and gives a .txt file of the news broadcast transcripts.

For basic usage of how to run colab, go to [official intro](https://colab.research.google.com/notebooks/intro.ipynb#recent=true).

# Install required modules and mount drive data

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
!pip install Pillow
!pip install Selenium-Screenshot
!pip install colorama





Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,194 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,420 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,274 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,466 kB]
Hit:13 https://ppa.launchpadcontent.n

In [ ]:
import time
import os
import pandas as pd
import sys
import pathlib
from itertools import product

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, UnexpectedAlertPresentException, ElementClickInterceptedException

from colorama import Fore, Back, Style


# UPDATE 10/22/23
from PIL import Image
from Screenshot import Screenshot


In [ ]:
# mount Drive
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/Shareddrives/'Election Local News Project'/'NDS News Scraping [F23] 2019'/'News Transcripts'

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive
/content/drive/Shareddrives/Election Local News Project/NDS News Scraping [F23] 2019/News Transcripts


In [ ]:
# change this line to navigate to the desired location (optional)
# %cd /content/drive/Shareddrives/"Election Local News Project"/"Don't Touch: Floyd Protest, Chauvin Trial News Coverage MEDIA (video & text)"/"Chauvin Trial News Coverage"/"Chauvin Trial Transcripts"
# %cd "/content/drive/My Drive/LOCAL DATASQUAD/News Transcripts"


# Body of code

## unique_program.py


In [ ]:
# Given a xls file (nds_xls) downloaded from News Data Service, returns all the unique broadcasts in the xls as csv (programs_file).
# Last execution: Sep 2021, Google Colab

# convert "8:00 AM CT" to "08:00AM CT"
def format_time(time_str):
    if time_str[1] == ':':
        return_str = '0' + time_str
    else:
        return_str = time_str
    # convert "08:00 AM CT" to "08:00AM CT"
    return return_str[:5] + return_str[6:]

def get_unique_program(nds_xls, programs_file):
    programs = pd.read_excel(nds_xls)
    # the five columns together serve as a unique identifier for each program
    program_uni = programs.loc[:, ['Date', 'Time', 'Title', 'Source', 'Market']].drop_duplicates()
    # date from Jul 5 2020 -> 2020-07-05
    df = program_uni.assign(Date = pd.to_datetime(program_uni['Date'], format='%b %d %Y')).astype(str)
    df['Time'] = df['Time'].apply(format_time) # make time conversion
    df['Scraped'] = False # initialize 'Scraped' column to false
    df_sorted = df.sort_values(['Market', 'Source', 'Date', 'Time'])
    df_sorted.reset_index(drop=True, inplace=True)
    df_sorted.to_csv(programs_file, index=False)

**9/4 by Helen: I added the "Scraped" column to programs_file in the get_unique_program() function. Before that I ran into an error that says programs_file does not have a "Scraped" column... I thought Hiro added that column to programs_file in nds_crawler() since that's what he said on the README, but I checked the code and it really did not add anything to programs_file. Maybe Hiro meant to say urls_file but he typed programs_file instead, on the README.**



## nds_crawler.py


For each unique combination of Date, Source, Market columns in programs_file csv,
make a query and write the results to csv (urls_file).
Last execution: Sep 2021, in Google Colab

**6/25 by Helen: While I was checking the XPath, I found something weird: for national stations, there are two possible options to choose from in the States box and the Cities box: "National News Networks" and "National Networks (All Other)". The national stations we want (ABC, CBS, CNN, FOX, FOXNEWS, MSNBC, NBC) can be found under both options, except for PBS, which can only be found under "National Networks (All Other)". However, for the 7 stations that can be searched using both options, the search results are different... I have only tried to compare the search results for ABC using the two different options, and it seems like the search results under "National Networks (All Other)" is a subset of the search results under "National News Networks". But I'm not sure if it is the same situation for other stations. For now, I think we should use "National News Networks" (in both the States and Cities boxes) for all national stations except for PBS. We can try this first, and if there are failed queries, we can deal with those later...**

**10/23/23 by Aaron: CCX was missing from the old hard coded "station_to_num" dictionary, which caused some indexing issues and improper scraping of local news sources. We added CCX in at position 2.**

# New Section

In [ ]:
# This function takes in the name of the market as input, and returns the x_path of the state and city as output.
# remember to go to NDS to check whether the x-path is right or not, since NDS may change the order of the Market name.
# To be more specific, you should check, for example, the number (5, in this case) in "//*[@id="states_listbox"]/li[5].
# NOTE: xpath indexes start at 1, not 0. So the number in li[] should be the data-offset-index plus 1
def return_Market_xpath(Market):
    if Market == 'Minneapolis-St. Paul, MN':
        return ['//*[@id="states_listbox"]/li[5]', '//*[@id="cities_listbox"]/li[3]']

    elif Market == 'Madison, WI':
        return ['//*[@id="states_listbox"]/li[7]', '//*[@id="cities_listbox"]/li[4]']

    elif Market == 'Cedar Rapids-Waterloo-Dubuque, IA':
        return ['//*[@id="states_listbox"]/li[4]', '//*[@id="cities_listbox"]/li[1]']

    elif Market == 'National News Networks':
        # note: National News Networks does not contain PBS
        return ['//*[@id="states_listbox"]/li[1]', '//*[@id="cities_listbox"]/li[1]']

    elif Market == 'National Networks (All Other)':
        # for PBS
        return ['//*[@id="states_listbox"]/li[2]', '//*[@id="cities_listbox"]/li[1]']

    else:
        raise Exception(f"Market name {Market} is not valid")


# This function takes in the name of the station as input, and returns the x_path of the station as output.
# remember to go to NDS to check whether the x-path is right or not, since NDS may change the order of the source name.
# for all states/cities except for National Networks (All Other)
def return_Source_xpath_local(Source):

    station_to_num = {

        # Local news stations, with their location on the
        # drop down menu
        'CBSN Minneapolis': '1',
        'CCX' : '2',
        'KARE': '3',
        'KCCO-AM': '4',
        'KDWB-FM': '5',
        'KEEY-FM': '6',
        'KFAN-FM': '7',
        'KFXN-FM': '8',
        'KLTF-AM': '9',
        'KMSP': '10',
        'KNOW-FM': '11',
        'KNSI-AM': '12',
        'KOWZ-FM': '13',
        'KQQL-FM': '14',
        'KQRS-FM': '15',
        'KSTC': '16',
        'KSTP': '17',
        'KSTP-AM': '18',
        'KTCA': '19',
        'KTCI': '20',
        'KTCZ-FM': '21',
        'KTLK-AM': '22',
        'KTMY-FM': '23',
        'KTWN-FM': '24',
        'TPT': '25',
        'WCCO': '26',
        'WCCO-AM': '27',
        'WFTC': '28',
        'WMNN-AM': '29',
        'WUCW': '30',
        'WUMN': '31',
        'WVAL-AM': '32',
    }

    return '//*[@id="sources_listbox"]/li[' + station_to_num[Source] + ']'



def return_Source_xpath_national_all_other(source):
    station_to_num = {

        # National news (All Other) stations, with their location on the
        # drop down menu
        'ABC' : '3',
        'ABC News': '4',
        'CBS' : '43',
        'CBS News': '44',
        'CNN' : '62',
        'FOX' : '127',
        'FOXNEWS' : '134',
        'MSNBC' : '200',
        'NBC' : '211',
        'NBC News Now': '212',
        'PBS': '239',

    }

    return '//*[@id="sources_listbox"]/li[' + station_to_num[source] + ']'
    # return '//*[@id="sources_listbox"]/li[text()=' + Source + ']'


def return_Source_xpath_national_news_networks(source):
    station_to_num = {
        'ABC': '1',
        'CBS': '5',
        'CNN': '7',
        'FOX' : '14',
        'FOXNEWS': '15',
        'MSNBC': '17',
        'NBC': '18',
        # No PBS in National News Networks - it's only in all-other
    }
    return f'//*[@id="sources_listbox"]/li[{ station_to_num[source] }]'



In [ ]:
def nds_crawl(username: 1,password: 1,programs_file: 1,urls_file,urls_file_100, crawled_file, failed_query_file, driver_option):
    Programs_List = pd.read_csv(programs_file)
    # The three columns identify exactly what is needed to make a query in NDS
    Query_List = Programs_List.loc[:, ['Date', 'Source', 'Market']].drop_duplicates().reset_index(drop=True)

    # If urls_file already exists, read it and delete the queries that are already done from Query_List
    # BUG: if urls_file contains a query that Query_List does not, that new query gets added
    # if pathlib.Path(urls_file).is_file():
    #   df_urls = pd.read_csv(urls_file)
    #   done_Query_List = df_urls.loc[:, ['Date', 'Source', 'Market']].drop_duplicates().reset_index(drop=True)
    #   Query_List = pd.concat([Query_List, done_Query_List]).drop_duplicates(keep=False).reset_index(drop=True)
    #   print("Query list")
    #   print(Query_List)

    print(Query_List)

    links = [] # the list to store all the URLs
    successful_query = []
    failed_query = []
    query_idx = 0
    num_query = Query_List.shape[0]
    print("Making {} total queries".format(num_query))

    while query_idx < num_query: # query is not completed
        Driver_Success = True # Driver has not encounterd a fatal failure

        # (re)start the webdriver in background
        driver = webdriver.Chrome(options=driver_option)
        # login
        driver.get('https://portal.newsdataservice.com/ProgramList')

        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "Loginform"))
        )

        driver.find_element(by=By.XPATH, value='//*[@id="Loginform"]/div/div[2]/div/p[3]/input').click()
        driver.find_element(by=By.XPATH, value='//*[@id="Loginform"]/div/div[2]/div/p[3]/input').send_keys(username)
        driver.find_element(by=By.XPATH, value='//*[@id="Loginform"]/div/div[2]/div/p[5]/input').click()
        driver.find_element(by=By.XPATH, value='//*[@id="Loginform"]/div/div[2]/div/p[5]/input').send_keys(password)
        driver.find_element(by=By.XPATH, value='//*[@id="submitBtn"]').click()
        print("Logging in to new NDS instance...")

        # continue with the same driver while a query remains to be conducted
        # and the driver has not encounterd any fatal failure
        first_query = True
        while Driver_Success and (query_idx < num_query):
            row = Query_List.loc[query_idx,]
            Market = row['Market']
            Source = row['Source']
            Date_str = row['Date']
            # a list to store a program by its identification and its url
            programs = []
            print(Fore.BLACK)
            print(Back.GREEN)
            print(f"=== Query {query_idx}, which is for market {Market}, source {Source}, and date {Date_str} ===")
            print(Style.RESET_ALL)

            # defining fields to select
            Market_x_path = return_Market_xpath(Market)
            State_x_path = Market_x_path[0]
            City_x_path = Market_x_path[1]

            Source_x_path = ''
            if Market == 'National News Networks':
                Source_x_path = return_Source_xpath_national_news_networks(Source)
            elif Market == 'National Networks (All Other)':
                Source_x_path = return_Source_xpath_national_all_other(Source)
            else:
                Source_x_path = return_Source_xpath_local(Source)

            # Source_x_path = f"*[@id='sources_listbox']/li[. = '{Source}']"

            num_failed_attempt = 0
            Submit_Timeout = False
            Query_Success = False # status of most recent attempt of this particular query
            # until we succeed or make 3 failed attepts
            Added_to_Failed = False

            while (not Submit_Timeout) and (not Query_Success) and (num_failed_attempt < 3):
                # (re)start the attempt

                try:
                    if first_query:
                      # navigating to search
                      print("Finding the Broadcast Content tab")
                      nav_button = WebDriverWait(driver, 20).until(
                          EC.element_to_be_clickable((By.XPATH, '//*[@id="navigation"]/li[6]/a'))
                      )
                      #time.sleep(7)
                      #By.XPATH, '/html/div[2]'
                      WebDriverWait(driver, 10).until(
                          EC.invisibility_of_element_located((By.CLASS_NAME, 'js-spin-overlay'))
                      )
                      time.sleep(1)
                      first_query = False
                      print('Clicking on the Broadcast Content tab')
                      nav_button.click()
                      #time.sleep(7)
                      WebDriverWait(driver, 10).until(
                          EC.invisibility_of_element_located((By.CLASS_NAME, 'js-spin-overlay'))
                      )
                    else:
                      print("Clearing search boxes")
                      WebDriverWait(driver, 10).until(
                          EC.invisibility_of_element_located((By.CLASS_NAME, 'js-spin-overlay'))
                      )

                      WebDriverWait(driver, 30).until(
                          EC.element_to_be_clickable((By.XPATH, '//*[@id="clearAllStates"]'))).click()
                      WebDriverWait(driver, 10).until(
                            EC.invisibility_of_element_located((By.CLASS_NAME, 'js-spin-overlay'))
                        )

                    # clicking on states box
                    print("Finding states box")
                    states_box = WebDriverWait(driver, 20).until(
                        EC.element_to_be_clickable((By.XPATH, '//*[@id="states_taglist"]'))
                    )
                    print("Clicking states box")
                    states_box.click()

                    #time.sleep(7)
                    WebDriverWait(driver, 10).until(
                        EC.invisibility_of_element_located((By.CLASS_NAME, 'js-spin-overlay'))
                    )

                    # selecting state
                    print("Selecting the state (or national)")
                    state_selection = WebDriverWait(driver, 20).until(
                        EC.element_to_be_clickable((By.XPATH, State_x_path))
                    )
                    print('Clicking the state selection')
                    state_selection.click()
                    #time.sleep(7)
                    WebDriverWait(driver, 10).until(
                        EC.invisibility_of_element_located((By.CLASS_NAME, 'js-spin-overlay'))
                    )

                    # clicking on cities box
                    cities_box = WebDriverWait(driver, 20).until(
                        EC.element_to_be_clickable((By.XPATH, '//*[@id="cities_taglist"]'))
                    )
                    print("Clicking on the cities filter box")
                    cities_box.click()
                    # time.sleep(7)
                    WebDriverWait(driver, 10).until(
                        EC.invisibility_of_element_located((By.CLASS_NAME, 'js-spin-overlay'))
                    )
                    # states_box.click()
                    # driver.find_element(by=By.XPATH, value='//*[@id="powerCities"]/div[4]/div/input').click()

                    # selecting city
                    print("Selecting the city")
                    WebDriverWait(driver, 30).until(
                        EC.element_to_be_clickable((By.XPATH, City_x_path))
                    ).click()
                    # driver.find_element(by=By.XPATH, value=City_x_path).click()
                    # time.sleep(10)
                    print("Loading...")
                    WebDriverWait(driver, 10).until(
                        EC.invisibility_of_element_located((By.CLASS_NAME, 'js-spin-overlay'))
                    )


                    # clicking on source box
                    print("Clicking on the source box")

                    WebDriverWait(driver, 20).until(
                        EC.element_to_be_clickable((By.XPATH, '//*[@id="sources_taglist"]'))
                    ).click()

                    # driver.find_element(by=By.XPATH, value='//*[@id="srcWrapper"]/div/div').click()
                    # time.sleep(10)
                    WebDriverWait(driver, 10).until(
                        EC.invisibility_of_element_located((By.CLASS_NAME, 'js-spin-overlay'))
                    )

                    # selecting source
                    print("Selecting the source")
                    WebDriverWait(driver, 20).until(
                        EC.element_to_be_clickable((By.XPATH, Source_x_path))
                    ).click()
                    # driver.find_element(by=By.XPATH, value=Source_x_path).click()
                    # time.sleep(7)
                    WebDriverWait(driver, 10).until(
                        EC.invisibility_of_element_located((By.CLASS_NAME, 'js-spin-overlay'))
                    )

                    # clicking on date box
                    print("Clicking on date box")
                    WebDriverWait(driver, 20).until(
                        EC.element_to_be_clickable((By.XPATH, '//*[@id="datePrograms"]'))
                    ).click()
                    # driver.find_element(by=By.XPATH, value='//*[@id="datePrograms"]').click()
                    # time.sleep(7)
                    WebDriverWait(driver, 10).until(
                        EC.invisibility_of_element_located((By.CLASS_NAME, 'js-spin-overlay'))
                    )

                    # clearing default date from date box
                    print("Clearing date box")
                    date_box = WebDriverWait(driver, 20).until(
                        EC.element_to_be_clickable((By.XPATH, '//*[@id="datePrograms"]'))
                    )
                    date_box.clear()
                    # driver.find_element(by=By.XPATH, value='//*[@id="datePrograms"]').clear()
                    # time.sleep(5)
                    WebDriverWait(driver, 10).until(
                        EC.invisibility_of_element_located((By.CLASS_NAME, 'js-spin-overlay'))
                    )

                    # sending date
                    print(f"Entering desired date {Date_str} into date box")
                    date_box.send_keys(Date_str)
                    # driver.find_element(by=By.XPATH, value='//*[@id="datePrograms"]').send_keys(Date_str)
                    # time.sleep(7)
                    WebDriverWait(driver, 10).until(
                        EC.invisibility_of_element_located((By.CLASS_NAME, 'js-spin-overlay'))
                    )

                    # UPDATE 10/22/23
                    ob = Screenshot.Screenshot()
                    img_url = ob.full_screenshot(driver, save_path=r'.', image_name='myimage.png', is_load_at_runtime=True,load_wait_time=3)


                    # clicking submit
                    print('Finding submit button')
                    # submit_button = driver.find_element(by=By.XPATH, value='//*[@id="btnListPrograms"]')
                    # submit_button = driver.find_element(By.ID, "btnListPrograms")
                    submit_button = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, 'btnListPrograms')))

                    print('found submit button')
                    time.sleep(1)
                    # driver.execute_script("document.getElementById('btnListPrograms').click();")
                    submit_button.click()
                    print("submit complete")
                    time.sleep(1)


                    # WebDriverWait(driver, 10).until(
                    #     EC.presence_of_element_located((By.ID, 'js-spin-spinner'))
                    # )
                    try:
                      WebDriverWait(driver, 120).until(
                          EC.invisibility_of_element_located((By.XPATH, '/html/div[2]'))
                      )
                    except Exception as e:
                      print(e)
                      print("Submit timeout occurred, not attempting query again")
                      successful_query.append([Market, Source, Date_str])
                      Submit_Timeout = True
                      Query_Success = False


                    found = False
                    if not Submit_Timeout:
                      print("looking for transcripts")
                      try:
                          # We want to detect the message box when it is
                          # <div class="success" style="display: block;">No programs found.</div>
                          # but not:
                          # <div class="success" style="display: none;">No programs found.</div>
                          WebDriverWait(driver, 20).until(
                              EC.presence_of_element_located((By.XPATH,'//div[@class="success" and @style="display: block;"]'))
                          )

                          print("no transcripts found")
                          found = False
                          num_failed_attempt = 3
                          print(f"Attempt Failed: query #{query_idx} attempt #{num_failed_attempt}")
                          print("Writing successful query (regardless of no transcripts found) to drive")
                          df_successful = pd.DataFrame(successful_query, columns=['Market','Source', 'Date'])
                          if pathlib.Path(crawled_file).is_file(): # append rows without header if failed_query_file already exists
                              df_successful.to_csv(crawled_file, index=False, mode='a', header=False)
                          else:
                              df_successful.to_csv(crawled_file, index=False)

                      except UnexpectedAlertPresentException as alert_exception:
                          # Handle the unexpected alert
                          print(f"Unexpected alert: {alert_exception}")
                          # You can choose to accept, dismiss, or perform other actions with the alert here.
                          # For example, to accept the alert:
                          driver.switch_to.alert.accept()
                      except TimeoutException as e:
                          found = True

                    if found:
                        # Wait until the first row of the results table appears
                        # Note that the NDS website seems to create the table itself before it fills
                        # in the rows, so we can't rely on the tbody's existence to tell us
                        # when the results have loaded
                        print('Waiting for results to load...')
                        results_table = WebDriverWait(driver, 120).until(
                            EC.presence_of_element_located((By.XPATH, '//*[@id="results"]/table/tbody/tr[1]')),
                        )
                        print('Results have loaded or else it timed')

                        # collect all urls from the query
                        next_row_exists = True

                        i = 0 #Note that XPath indexing starts at 1
                        while next_row_exists:
                            i = i + 1
                            try:
                                # extract link
                                xpath = '//*[@id="results"]/table/tbody/tr[' + str(i) + ']/td[5]/a'
                                onclick = driver.find_element(by=By.XPATH, value=xpath).get_attribute('onclick')
                                url = onclick.split("'")[1]

                                # extract time and title of the program (from the web page listing search results) with the url
                                Time_str = driver.find_element(by=By.XPATH, value='//*[@id="results"]/table/tbody/tr[' + str(i) + ']/td[3]').text
                                Title_str = driver.find_element(by=By.XPATH, value='//*[@id="results"]/table/tbody/tr[' + str(i) + ']/td[4]').text

                                # attach the program identification and URL to the nested list
                                programs.append([Date_str, Time_str, Title_str, Source, Market, url, False])
                            except:
                                # if the next link does not exist, set b to false and exit query
                                next_row_exists = False

                        print("num links collected: " + str(len(programs)))
                        # time.sleep(3)

                        Query_Success = True # the query attempt ran without error
                        if len(programs) > 0:
                            links = links + programs # add the programs from this success query
                            #print("APPENDING" +str(Market)+", "+str(Source)+", "+str(Date_str) +" to succeded queries...")
                            successful_query.append([Market, Source, Date_str])
                        else:
                            #print("APPENDING" +str(Market)+", "+str(Source)+", "+str(Date_str) +" to failed queries...")
                            failed_query.append([Date_str, Source, Market, 'no link']) # no program is found

                except Exception as e: # the query attempt has failed
                    template = "An exception of type {0} occurred. Arguments:\n{1!r}"
                    message = template.format(type(e).__name__, e.args)
                    print(Back.RED + Fore.WHITE + message)
                    print(Style.RESET_ALL)
                    ob = Screenshot.Screenshot()
                    img_url = ob.full_screenshot(driver, save_path=r'.', image_name='error_debug.png', is_load_at_runtime=True,load_wait_time=3)
                    # print(e)

                    num_failed_attempt += 1
                    print(f"Attempt Failed: query #{query_idx} attempt #{num_failed_attempt}")

            if not Query_Success: # all attempts failed
                Driver_Success = False # encountered a fatal failure
                if query_idx == num_query:
                  failed_query.append([Date_str, Source, Market, '3 failed attempts'])
                else:
                  if Submit_Timeout:
                    failed_query.append([Date_str, Source, Market, 'submission timeout occurred'])
                    # print("Writing successful query due to submit timeout")
                    # df_successful = pd.DataFrame(successful_query, columns=['Market','Source', 'Date'])
                    # if pathlib.Path(crawled_file).is_file(): # append rows without header if failed_query_file already exists
                    #     df_successful.to_csv(crawled_file, index=False, mode='a', header=False)
                    # else:
                    #     df_successful.to_csv(crawled_file, index=False)
                  else:
                    failed_query.append([Date_str, Source, Market, 'no transcripts found'])
                print(Back.RED + Fore.WHITE + f"Query {query_idx} has failed. Quitting and restarting webdriver")
                print(Style.RESET_ALL)
                driver.quit()

            print("Writing URLS to drive...")
            df_urls = pd.DataFrame(links, columns=['Date', 'Time', 'Title', 'Source', 'Market', 'URL', 'Scraped']).drop_duplicates().reset_index(drop=True)
            if pathlib.Path(urls_file).is_file(): # append rows without header if urls_file already exists
                df_urls.to_csv(urls_file, index=False, mode='a', header=False)
            else:
                df_urls.to_csv(urls_file, index=False)

            print("Writing top 100 URLs to drive...")
            if pathlib.Path(urls_file_100).is_file(): # append rows without   if urls_file already exists for each 100 rows
                df_urls.to_csv(urls_file_100, index=False)
            else:
                df_urls.to_csv(urls_file_100,index = False)

            # Writing successful queries
            print("Writing successful query (if any) to drive")
            df_successful = pd.DataFrame(successful_query, columns=['Market','Source', 'Date'])
            if pathlib.Path(crawled_file).is_file(): # append rows without header if failed_query_file already exists
                df_successful.to_csv(crawled_file, index=False, mode='a', header=False)
            else:
                df_successful.to_csv(crawled_file, index=False)

            # writing queries that encountered a fatal failure
            print("Writing failed query (if any) to drive")
            df_failed = pd.DataFrame(failed_query, columns=['Date', 'Source', 'Market', 'Error'])
            if pathlib.Path(failed_query_file).is_file(): # append rows without header if failed_query_file already exists
                df_failed.to_csv(failed_query_file, index=False, mode='a', header=False)
            else:
                df_failed.to_csv(failed_query_file, index=False)
            links.clear()
            successful_query.clear()
            failed_query.clear()

            query_idx += 1 # move on to the next query

    driver.quit()
    print("All queries complete")

    # write obtained URLs to a csv file
    # Note: Date & Source & Market is from programs_file, Time & Title & URL is from the web page listing search results
    # df_urls = pd.DataFrame(links, columns=['Date', 'Time', 'Title', 'Source', 'Market', 'URL', 'Scraped']).drop_duplicates().reset_index(drop=True)
    # if pathlib.Path(urls_file).is_file(): # append rows without header if urls_file already exists
    #     df_urls.to_csv(urls_file, index=False, mode='a', header=False)
    # else:
    #     df_urls.to_csv(urls_file, index=False)

    # if pathlib.Path(urls_file_100).is_file(): # append rows without header if urls_file already exists for each 100 rows
    #     df_urls.to_csv(urls_file_100, index=False)
    # else:
    #     df_urls.to_csv(urls_file_100,index = False)

    # # writing queries that encountered a fatal failure
    # df_failed = pd.DataFrame(failed_query, columns=['Date', 'Source', 'Market', 'Error'])
    # if pathlib.Path(failed_query_file).is_file(): # append rows without header if failed_query_file already exists
    #     df_failed.to_csv(failed_query_file, index=False, mode='a', header=False)
    # else:
    #     df_failed.to_csv(failed_query_file, index=False)

## select_urls.py


In [ ]:
# Given all the links from a query (urls_file) and the list of required programs (programs_file),
# returns a csv file of just the required urls
# Last execution: Sep 2021, Google Colab

# parse url to get time and title of the program
def get_Time_and_Title(nds_url_str):
    prog_datetime = nds_url_str.split('&')[2].split('=')[-1]
    prog_title = nds_url_str.split('&')[4].split('=')[-1].replace('%20', ' ').replace('%40', '@')
    return prog_datetime[11:], prog_title


def select_urls(urls_file, programs_file):
    # df_prog has column names Date, Time, Title, Source, Market, URL, Scraped in this order
    df_prog = pd.read_csv(programs_file)
    df_url = pd.read_csv(urls_file)
    # sort two dataframes in the same order
    for df in [df_prog, df_url]:
        df.sort_values(['Market', 'Source', 'Date', 'Time'], inplace=True, ignore_index=True)

    # insert missing time and title (extracted from the url) if one of them is missing
    df_missing = df_url.loc[df_url['Time'].isnull() | df_url['Title'].isnull()]
    for ind, row in df_missing.iterrows():
        prog_time, prog_title = get_Time_and_Title(row['URL'])
        df_url.loc[ind, 'Time'] = prog_time
        df_url.loc[ind, 'Title'] = prog_title

    df_prog['URL'] = ''

    search_start = 0
    for i in range(df_prog.shape[0]):
        ith_row_id = df_prog.loc[i, ['Date', 'Time', 'Title', 'Source', 'Market']].values
        for j in range(df_url.shape[0]):
            jth_row_id = df_url.loc[j, ['Date', 'Time', 'Title', 'Source', 'Market']].values
            if all(ith_row_id == jth_row_id):
                df_prog.loc[i, 'URL'] = df_url.loc[j, 'URL'] #copy url
                break

    df_prog.to_csv(programs_file, index=False)
    df_url.to_csv(urls_file, index=False)


# use this function instead of select_urls() if need non-program-specific transcripts
def fill_in_missing_time_title(urls_file):
  df_url = pd.read_csv(urls_file)
  df_missing = df_url.loc[df_url['Time'].isnull() | df_url['Title'].isnull()]
  for ind, row in df_missing.iterrows():
      prog_time, prog_title = get_Time_and_Title(row['URL'])
      df_url.loc[ind, 'Time'] = prog_time
      df_url.loc[ind, 'Title'] = prog_title
  df_url.to_csv(urls_file, index=False)

## nds_scraper.py

In [ ]:
# This file is used to iterate through each url,
# scrape the text within the url, and store it in the directory we want.
# Last execution: Sep 2021, Google Colab

In [ ]:
# format the date for filename
def return_Date_str(Date):

    monthToNum = {
        'JAN' : '1',
        'FEB' : '2',
        'MAR' : '3',
        'APR' : '4',
        'MAY' : '5',
        'JUN' : '6',
        'JUL' : '7',
        'AUG' : '8',
        'SEP' : '9',
        'OCT' : '10',
        'NOV' : '11',
        'DEC' : '12'
    }

    parse_date = Date.split(" ")

    month = monthToNum[parse_date[0]]
    date = parse_date[1]
    year = parse_date[2]

    return year + "-" + month + "-" + date

In [ ]:
def nds_scrape(programs_file, unscraped_programs_file, transcripts_dir, driver_option):
    df = pd.read_csv(programs_file)
    df["Error"] = ''
    driver = webdriver.Chrome(options=driver_option)

    print("scrape total of {} indices".format(df.shape[0]))
    df = df.drop_duplicates(subset = "URL")

    #df = df[df.Source == 'ABC News']
    # iterate through all of the links
    for index, row in df.iterrows():
        if row['URL'] == '':
            print("index {} has missing URL".format(index))
        elif row['Scraped']:
            print("index {} has already been scraped".format(index))
        else:
            try:
                print("scraping text from index {}".format(index))
                # go to link
                driver.get(row['URL'])
                # # extract text from the body of the website
                body = driver.find_element(By.XPATH, "/html/body/table/tbody").text
                # # split body of the text by line
                # split = body.splitlines()
                # # extract station
                # station = split[0].strip()
                # # extract date and broadcast
                # date_broadcast = split[1]
                # date_broadcast_split = date_broadcast.split("  ")
                # date = return_Date_str(date_broadcast_split[0]) #2008-7-28
                # broadcast = date_broadcast_split[1].strip()

                # # extract the first timestamp
                # first_paragraph = split[2].split(" ")
                # time = first_paragraph[0][3:]
                # time = time.replace(":", "_").strip()
                # am_pm = first_paragraph[1][:2].strip()

                # # save the text of the body to a .txt file in the specified directory
                # # filename = e.g. 'KARE 2008-7-28 04_00_02PM KARE 11 AT 4.txt'
                # filename = station + " " + date + " " + time + am_pm + " " + broadcast + ".txt"
                station, month, date, fileName = downloaded_file_name_formatter(row)
                fullDir = '{}/{}/{}/{}'.format(transcripts_dir, station, month, date)
                ensure_dir(fullDir) # ensure that the directories for the mp4 exist
                fullPath = '{}/{}/{}/{}/{}.txt'.format(transcripts_dir, station, month, date, fileName)
                # if fullPath already exists, raise an error
                assert (not pathlib.Path(fullPath).is_file()), "File '{}' already exists\n".format(fileName)


                # # specify directory
                # file_dir = '{}/{}'.format(transcripts_dir, station)
                # if not os.path.exists(file_dir):
                #     os.makedirs(file_dir)
                # filepath = os.path.join(file_dir, filename)
                file = open(fullPath, "w")
                file.write(body)
                file.close()
                df.loc[index, 'Scraped'] = True
            except Exception as e:
                print(e)
                print("failed to scrape text from index {}".format(index))
                df.loc[index, 'Error'] = e

        if (index + 1)%50==0: # update csv file after 50 iterations
            df_unscraped = df[~df['Scraped']] # programs that were not scraped
            df.to_csv(programs_file, index=False)
            df_unscraped.to_csv(unscraped_programs_file, index=False)

    df_unscraped = df[~df['Scraped']] # programs that were not scraped
    df.to_csv(programs_file, index=False)
    df_unscraped.to_csv(unscraped_programs_file, index=False)
    driver.quit()

In [ ]:
# checks if the path specified by file_dir is a existing directory, if not then create it (recursively)
def ensure_dir(file_dir):
    if not pathlib.Path(file_dir).is_dir():
        pathlib.Path(file_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
# format parts of the file path to match the existing folders & files:
# currently, path would be station/month/date/fileName
def downloaded_file_name_formatter(row):
    station = row['Source']

    monthNum = row['Date'].split('-')[1]
    month_conversion = {'01':'January', '02':'February', '03':'March', '04':'April', '05':'May', '06':'June', '07':'July', '08':'August', '09':'September', '10':'October', '11':'November', '12':'December'}
    assert (monthNum in month_conversion),  ('Error reading program month:' + month + 'read\n')
    month = month_conversion.get(monthNum)

    # date example: 042321
    year = row['Date'].split('-')[0][2:]
    day = row['Date'].split('-')[2]
    date = monthNum + day + year
    # fileName example: FOX News 041421 400PM CT_1 (the "_1" means part 1)

    # first read the part number of the program title (may be empty)
    partNum = ""
    # ptSubstring_index is the index of the substring 'pt' in the program name
    ptSubstring_index = row['Title'].find('pt', -5, len(row['Title']))
    if ptSubstring_index != -1:
      # both 'pt.3' and 'pt 3' can appear in program names
      partNum = row['Title'][ptSubstring_index + 3]
      # deal with 'pt. 3'
      if partNum == " ":
        partNum = row['Title'][ptSubstring_index + 4]
      assert partNum.isnumeric(), ('Error reading program part number:' + partNum + 'read\n')
      partNum = '_' + partNum

    time_raw = row['Time'].split()
    timeZone = ""
    try:
      time = time_raw[0]
      timeZone = time_raw[1]
    except Exception as e:
      print("An error occured while fetching the time zone: ",e)
    fileName = " ".join([station, date, time+' '+timeZone+partNum])

    return station, month, date, fileName

# Run the main function

## nds_main.py

**Create programs_file using dataframe:**

In [ ]:
# local_sources = [
#     'CBSN Minneapolis', 'KARE', 'KCCO-AM', 'KDWB-FM', 'KEEY-FM',
#     'KFAN-FM', 'KFXN-FM','KLTF-AM','KMSP','KNOW-FM',
#     'KNSI-AM', 'KOWZ-FM','KQQL-FM', 'KQRS-FM', 'KSTC',
#     'KSTP','KST-AM','KTCA', 'KTCI','KTCZ-FM',
#     'KTLK-AM', 'KTMY-FM','KTWN-FM','TPT','WCCO',
#     'WCCO-AM','WFTC','WMNN-AM','WUCW', 'WUMN',
#     'WVAL-AM'
# ]

local_sources = ['WCCO', 'KARE', 'KMSP', 'KSTP', 'TPT']


national_news_network_sources = [
    'ABC', 'CBS', 'CNN', 'FOX', 'FOXNEWS', 'MSNBC', 'NBC'
]
national_news_all_other_sources = [
    'ABC', 'ABC News', 'CBS', 'CBS News', 'CNN', 'FOX', 'FOXNEWS', 'MSNBC',
    'NBC', 'NBC News Now', 'PBS'
]

# local_sources = ['KSTP']

# national_news_network_sources = []

# national_news_all_other_sources = []


In [ ]:

# UPDATED BY AARON BRONSTONE 10/13/2023
date_range1 = list(pd.date_range(
    start='08/26/2004',
    end='08/27/2004'
))

date_range2 = list(pd.date_range(
    start='11/01/2019',
    end='12/31/2019'
))

date_range3 = list(pd.date_range(
    start='10/13/2004',
    end='10/17/2004'
))

date_range4 = list(pd.date_range(
    start='10/30/2004',
    end='11/1/2004'
))

#date_range = date_range1 + date_range2 + date_range3 + date_range4
date_range = date_range2


# Create a dataframe of the cartesian product of sources and dates
# aka each unique combination of the two

df_local_networks = pd.DataFrame(
    product(['Minneapolis-St. Paul, MN'], local_sources, date_range),
    columns=['Market', 'Source', 'Date']
)

df_national_national_networks = pd.DataFrame(
    product(['National News Networks'], national_news_network_sources, date_range),
    columns=['Market', 'Source', 'Date']
)

df_national_all_other_networks = pd.DataFrame(
    product(['National Networks (All Other)'], national_news_all_other_sources, date_range),
    columns = ['Market', 'Source', 'Date']
)


df_national_combined = pd.concat([df_national_national_networks, df_national_all_other_networks])
df_all = pd.concat([df_national_combined, df_local_networks])
df_all = df_all.reset_index(drop=True)

df_all


,Market,Source,Date
0,National News Networks,ABC,2019-11-01
1,National News Networks,ABC,2019-11-02
2,National News Networks,ABC,2019-11-03
3,National News Networks,ABC,2019-11-04
4,National News Networks,ABC,2019-11-05
...,...,...,...
1398,"Minneapolis-St. Paul, MN",TPT,2019-12-27
1399,"Minneapolis-St. Paul, MN",TPT,2019-12-28
1400,"Minneapolis-St. Paul, MN",TPT,2019-12-29
1401,"Minneapolis-St. Paul, MN",TPT,2019-12-30


In [ ]:
# options to make webdriver run in the background
op = Options()
op.add_argument("--disable-gpu")
op.add_argument("--disable-extensions")
op.add_argument("--proxy-server='direct://'")
op.add_argument("--proxy-bypass-list=*")
# op.add_argument("--start-maximized")
op.add_argument("--headless")
op.add_argument('-no-sandbox')
op.add_argument('-disable-dev-shm-usage')
op.add_argument("--window-size=1920,1080")


username = 'carletonba'
password = 'carletonba'

#MODIFY FILEPATHs

#CHANGE THIS
nds_xls = './programs_1100_1209.csv' # Excel file downloaded from NDS
programs_file = './programs_1100_1209.csv' # csv file to store programs

#CHANGE THIS
urls_file_100 = './scraped_transcripts_1100_1209.csv'

#Don't Change this
crawled_file = './crawled_queries.csv' # csv file to store all successful queries that have already been crawled
urls_file = './scraped_transcripts.csv' # csv file to store all the urls from queries related to each program
failed_query_file = './failed_query.csv' # csv file to write which queries have encountered fatal error
unscraped_programs_file = './unscraped.csv' # csv file to store programs that need to be scraped.
transcripts_dir = '.'

if 'REPLACE_THIS' in [username, password]:
    sys.exit('Please provide valid NDS login information.')

In [ ]:
df_sorted = df_all

# # An example data frame where NDS has no transcripts
# df_sorted = pd.DataFrame({
#     'Market': ['National Networks (All Other)'] ,
#     'Source': ['CNN'],
#     'Date': ['05/29/2020']
# })


df_sorted['Scraped'] = False
df_sorted = df_sorted.sort_values(['Market', 'Source', 'Date']).reset_index(drop=True)
df_sorted.reset_index(drop=True, inplace=True)

# Remove duplicates from failed query file
df_failed = pd.read_csv(failed_query_file)
df_failed.drop_duplicates()
df_failed = df_failed.sort_values(['Market', 'Source', 'Date']).reset_index(drop=True)
df_failed.to_csv(failed_query_file, index=False)

if pathlib.Path(crawled_file).is_file():
  print("CRAWLED FILE FOUND")
  df_already_crawled = pd.read_csv(crawled_file)
  df_already_crawled.drop_duplicates()
  df_already_crawled['Date'] = pd.to_datetime(df_already_crawled['Date'])
  print(df_sorted.dtypes)
  print(df_already_crawled.dtypes)
  df_already_crawled = df_already_crawled.sort_values(['Market', 'Source', 'Date']).reset_index(drop=True)
  df_already_crawled.to_csv(crawled_file, index=False)

  columns_to_compare = ['Market','Source','Date']

  merged = df_sorted.merge(df_already_crawled, on=columns_to_compare, how='left', indicator=True)
  # print(df_sorted[0:30])
  # print(merged[0:30])
  # Filter the rows in df1 that do not have a match in df2
  non_matching_rows_df1 = df_sorted[merged['_merge'] == 'left_only'].copy()
  # print(non_matching_rows_df1)
  df_sorted = non_matching_rows_df1
  # Drop the indicator column '_merge' from the result
  #df_sorted = non_matching_rows_df1.drop(columns=['_merge'])

#500 - 1000
df_sorted_100 = df_sorted.iloc[0:]

df_sorted_100.to_csv('./programs_1100_1209.csv', index=False)
df_sorted_100

CRAWLED FILE FOUND
Market             object
Source             object
Date       datetime64[ns]
Scraped              bool
dtype: object
Market            object
Source            object
Date      datetime64[ns]
dtype: object


<ipython-input-17-5b6eb941338f>:37: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  non_matching_rows_df1 = df_sorted[merged['_merge'] == 'left_only'].copy()


,Market,Source,Date,Scraped
38,"Minneapolis-St. Paul, MN",KARE,2019-12-09,False
62,"Minneapolis-St. Paul, MN",KMSP,2019-11-02,False
99,"Minneapolis-St. Paul, MN",KMSP,2019-12-09,False
131,"Minneapolis-St. Paul, MN",KSTP,2019-11-10,False
134,"Minneapolis-St. Paul, MN",KSTP,2019-11-13,False
...,...,...,...,...
1398,National News Networks,NBC,2019-12-27,False
1399,National News Networks,NBC,2019-12-28,False
1400,National News Networks,NBC,2019-12-29,False
1401,National News Networks,NBC,2019-12-30,False


In [ ]:
df_sorted_100

,Market,Source,Date,Scraped
38,"Minneapolis-St. Paul, MN",KARE,2019-12-09,False
62,"Minneapolis-St. Paul, MN",KMSP,2019-11-02,False
99,"Minneapolis-St. Paul, MN",KMSP,2019-12-09,False
131,"Minneapolis-St. Paul, MN",KSTP,2019-11-10,False
134,"Minneapolis-St. Paul, MN",KSTP,2019-11-13,False
...,...,...,...,...
1398,National News Networks,NBC,2019-12-27,False
1399,National News Networks,NBC,2019-12-28,False
1400,National News Networks,NBC,2019-12-29,False
1401,National News Networks,NBC,2019-12-30,False


**9/4 by Helen: added two options to the driver: no-sandbox and disable-dev-shm-usage. Honestly I have no idea what they mean and what OS Colab's server uses (some of these options are OS specific), but it seems to work... I also added transcripts_dir as an argument for nds_scrape, just to make it easier to identify & change where we're storing the transcripts.**


### Instructions:

1. **if you have the excel file from NDS, and you want to download transcripts for just the selected programs**,  run this cell：

In [ ]:
# This section keeps throwing errors. Is this code cell still relevant? 10/11/22
# get_unique_program(nds_xls, programs_file)
# nds_crawl(username, password, programs_file, urls_file, failed_query_file, driver_option=op)
# select_urls(urls_file, programs_file)
# nds_scrape(programs_file, unscraped_programs_file, transcripts_dir, driver_option=op)



2. **if you want to download all transcripts for specified Date & Source & Market, after making programs_file (by modifying the first code cell in nds_main.py, or manually creating a csv)**, run this cell:

In [ ]:
nds_crawl(username, password, programs_file, urls_file, urls_file_100, crawled_file, failed_query_file, driver_option=op)

           Date Source                    Market
0    2019-12-09   KARE  Minneapolis-St. Paul, MN
1    2019-11-02   KMSP  Minneapolis-St. Paul, MN
2    2019-12-09   KMSP  Minneapolis-St. Paul, MN
3    2019-11-10   KSTP  Minneapolis-St. Paul, MN
4    2019-11-13   KSTP  Minneapolis-St. Paul, MN
..          ...    ...                       ...
693  2019-12-27    NBC    National News Networks
694  2019-12-28    NBC    National News Networks
695  2019-12-29    NBC    National News Networks
696  2019-12-30    NBC    National News Networks
697  2019-12-31    NBC    National News Networks

[698 rows x 3 columns]
Making 698 total queries
Logging in to new NDS instance...


=== Query 0, which is for market Minneapolis-St. Paul, MN, source KARE, and date 2019-12-09 ===

Finding the Broadcast Content tab


KeyboardInterrupt: ignored

In [ ]:
fill_in_missing_time_title(urls_file)
nds_scrape(urls_file, unscraped_programs_file, transcripts_dir, driver_option=op)

scrape total of 651 indices
index 0 has already been scraped
index 1 has already been scraped
index 2 has already been scraped
index 3 has already been scraped
index 4 has already been scraped
index 5 has already been scraped
index 6 has already been scraped
index 7 has already been scraped
index 8 has already been scraped
index 9 has already been scraped
index 10 has already been scraped
index 11 has already been scraped
index 12 has already been scraped
index 13 has already been scraped
index 14 has already been scraped
index 15 has already been scraped
index 16 has already been scraped
index 17 has already been scraped
index 18 has already been scraped
index 19 has already been scraped
index 20 has already been scraped
index 21 has already been scraped
index 22 has already been scraped
index 23 has already been scraped
index 24 has already been scraped
index 25 has already been scraped
index 26 has already been scraped
index 27 has already been scraped
index 28 has already been scra

KeyboardInterrupt: ignored

# NEXT TO FIX/CONSIDER (NOT ranked by priority):
## 1 - Local stations: now have "AM/PM" options in SOURCES box: eg. "WCCO" vs "WCCO-AM"
## 2 - Find a way to avoid the need of separate rows for different dates in programs.csv
## 3 - National stations: "National News Network" vs "National News Network (All Other)" in STATES/CITIES box
## 4 - ~Find a way to NOT overwrite programs_file when we add more programs from new queries~

In [ ]:
if pathlib.Path(urls_file).is_file():
      df_urls = pd.read_csv(urls_file)
      done_Query_List = df_urls.loc[:, ['Date', 'Source', 'Market']].drop_duplicates().reset_index(drop=True)
      print(done_Query_List)

In [ ]:
Programs_List = pd.read_csv(programs_file)
# The three columns identify exactly what is needed to make a query in NDS
Query_List = Programs_List.loc[:, ['Date', 'Source', 'Market']].drop_duplicates().reset_index(drop=True)
print(Query_List)

In [ ]:
d3 = pd.concat([Query_List, done_Query_List]).drop_duplicates(keep=False)
print(d3)

In [ ]:
df = pd.read_csv(programs_file)
df['Scraped'] = False
df_sorted = df.sort_values(['Market', 'Source', 'Date'])
df_sorted.reset_index(drop=True, inplace=True)
df_sorted.to_csv('./programs.csv', index=False)
df

In [ ]:
df = pd.DataFrame([['2021-04-20', 'ABC', 'National News Networks'], ['2021-04-20', 'ABC', 'National Networks (All Other)']], columns=['Date', 'Source', 'Market'])
df['Scraped'] = False
df_sorted = df.sort_values(['Market', 'Source', 'Date'])
df_sorted.reset_index(drop=True, inplace=True)
df_sorted.to_csv('./programs.csv', index=False)
df



---



---




**Errors found as of Jun 3 2022:**   
```
delete date complete
Message: element not interactable
```
*   
```
select city complete
Message: element click intercepted: Element <div class="k-multiselect-wrap k-floatwrap" deselectable="on">...</div> is not clickable at point (120, 526). Other element would receive the click: <div class="js-spin-overlay" style="position: absolute; opacity: 0.5; z-index: 10009; background-color: rgb(0, 0, 0); left: 0px; top: 0px; width: 785px; height: 600px;"></div>
```
*
```
Message: element click intercepted: Element <a href="/ProgramList">...</a> is not clickable at point (236, 63). Other element would receive the click: <div class="js-spin-overlay" style="position: absolute; opacity: 0.5; z-index: 10009; background-color: rgb(0, 0, 0); left: 0px; top: 0px; width: 785px; height: 600px;"></div>
```





In [ ]:
nds_crawl(username, password, programs_file, urls_file, failed_query_file, driver_option=op)
fill_in_missing_time_title(urls_file)

In [ ]:
df = pd.read_csv(urls_file)
df2 = pd.read_csv(failed_query_file)
df2
# df = df.loc[:, ['Date', 'Time', 'Title', 'Source', 'URL']].drop_duplicates().reset_index(drop=True)
# df.shape[0]
# df
df2 = pd.read_csv(urls_file)
df2.sort_values(['Date', 'Time', 'Title', 'Market'], inplace=True, ignore_index=True)
df2.shape[0]
df2.to_csv('dup.csv')